In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
#alle urls und infos der Hauptseiten holen.
url = "https://kr-geschaefte.zug.ch/gast/geschaefte?commit=Filtern&geschaeft_filter%5Babgeschlossen_bis%5D=&geschaeft_filter%5Babgeschlossen_von%5D=&geschaeft_filter%5Barten_refs%5D%5B%5D=&geschaeft_filter%5Beingereicht_bis%5D=&geschaeft_filter%5Beingereicht_von%5D=&geschaeft_filter%5Bfrist_bis%5D=&geschaeft_filter%5Bhistorische_staende_refs%5D%5B%5D=&geschaeft_filter%5Bkommissionen_refs%5D%5B%5D=&geschaeft_filter%5Bstaende_refs%5D%5B%5D=&geschaeft_filter%5Bstatus_ids%5D%5B%5D=haengig&geschaeft_filter%5Bstatus_ids%5D%5B%5D=abgeschlossen&geschaeft_filter%5Bstatus_ids%5D%5B%5D=&geschaeft_filter%5Btitel%5D=&geschaeft_filter%5Bzustaendig_refs%5D%5B%5D=&page="

alle_seiten = []
for seite in range(1,2):
    r=requests.get(url+str(seite))
    soup = BeautifulSoup(r.text,"html.parser")
    
    tr_list=soup.find_all("tr")[2:] 
    
    for element in tr_list:
        td_list = element.find_all("td")
        vorlag_nr=td_list[0].text
        href = td_list[0].find("a")["href"]
        text = td_list[1].text
        stand = td_list[2].text
        zustaendig = td_list[3].text
        geschaeft_art = td_list[4].text
        status = td_list[5].text
        
        mini_dict = {"GeschäftsNR":vorlag_nr,"url":href,"Geschäft":text, "Stand":stand, "Zuständigkeit":zustaendig, "Art des Geschäfts": geschaeft_art, "Status":status}

        alle_seiten.append(mini_dict)



In [3]:
pd.DataFrame(alle_seiten)

,GeschäftsNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status
0,3016,/gast/geschaefte/2004,Gesetzesinitiative für längere Ladenöffnungsze...,Einreichung,,Initiative: Gesetzesinitiative,hängig
1,3010,/gast/geschaefte/1998,"Postulat von Ivo Egger, Stéphanie Vuichard, Ha...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig
2,3009,/gast/geschaefte/1997,"Postulat von Stéphanie Vuichard, Mariann Hess ...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig
3,3008,/gast/geschaefte/1996,Motion von Rainer Leemann und Michael Arnold b...,Überweisung an Regierungsrat,Gesundheitsdirektion,Motion,hängig
4,3007,/gast/geschaefte/1995,Interpellation der SP-Fraktion betreffend Pest...,Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig
5,3006,/gast/geschaefte/1994,Interpellation von Pirmin Andermatt und Heini ...,Überweisung an Regierungsrat,Finanzdirektion,Interpellation,hängig
6,3005,/gast/geschaefte/1993,Kantonsrats-Ersatzwahl in der Einwohnergemeind...,Wahl: Feststellung Gültigkeit,Staatskanzlei,Wahlen,abgeschlossen
7,3004,/gast/geschaefte/1992,Motion der CVP-Fraktion betreffend bedarfsgere...,Überweisung an Regierungsrat,Direktion für Bildung und Kultur,Motion,hängig
8,3003,/gast/geschaefte/1991,"Postulat von Manuela Leemann, Benny Elsener un...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig
9,3002,/gast/geschaefte/1990,"Interpellation von Hans Baumgartner, Manuela K...",Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig


In [4]:
df_a=pd.DataFrame(alle_seiten)

In [5]:
df_a["GeschäftsNR"]= df_a["GeschäftsNR"].astype(int) #die Geschäftsnummer (VorlagenNR) zu Zahl machen

In [6]:
url_liste_ganz=[]
url_anfang_original="https://kr-geschaefte.zug.ch"
for element in df_a["url"]:
    url_liste_ganz.append(url_anfang_original+element)
    

In [7]:
rg=requests.get("https://kr-geschaefte.zug.ch/gast/geschaefte/1997")
soup_g=BeautifulSoup(rg.text, "html.parser")
tr_g_list=soup_g.find_all("tr")

In [8]:
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
gesch_nr = tr_g_list[-1].find_all("td")[0].text
einger_am = tr_g_list[1].find_all("td")[0].text

minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am}

eingang_list=[]

for geschaeft in url_liste_ganz:
    rg=requests.get(geschaeft)
    soup_g=BeautifulSoup(rg.text, "html.parser")
    tr_g_list=soup_g.find_all("tr")
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text
      
    
    minidict_g={"GeschäftsNR":gesch_nr, "Einreichedatum":einger_am}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
            eingang_list.append(minidict_g)


In [9]:
df_g=pd.DataFrame(eingang_list)
pd.set_option("display.max_rows", 3000)

In [10]:
df_g["GeschäftsNR"]= df_g["GeschäftsNR"].astype(int) #verwandle string in Zahl
df_g["Einreichedatum"] =  pd.to_datetime(df_g["Einreichedatum"], format='%d.%m.%Y') #verwandle string in datum

In [11]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR") #ich verbinde die beiden DataFrames

In [12]:
df_mit_dat

,GeschäftsNR,url,Geschäft,Stand,Zuständigkeit,Art des Geschäfts,Status,Einreichedatum
0,3016,/gast/geschaefte/2004,Gesetzesinitiative für längere Ladenöffnungsze...,Einreichung,,Initiative: Gesetzesinitiative,hängig,2019-09-30
1,3016,/gast/geschaefte/2004,Gesetzesinitiative für längere Ladenöffnungsze...,Einreichung,,Initiative: Gesetzesinitiative,hängig,2019-09-30
2,3010,/gast/geschaefte/1998,"Postulat von Ivo Egger, Stéphanie Vuichard, Ha...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig,2019-09-10
3,3010,/gast/geschaefte/1998,"Postulat von Ivo Egger, Stéphanie Vuichard, Ha...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig,2019-09-10
4,3009,/gast/geschaefte/1997,"Postulat von Stéphanie Vuichard, Mariann Hess ...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig,2019-09-10
5,3009,/gast/geschaefte/1997,"Postulat von Stéphanie Vuichard, Mariann Hess ...",Überweisung an Regierungsrat,Baudirektion,Postulat,hängig,2019-09-10
6,3008,/gast/geschaefte/1996,Motion von Rainer Leemann und Michael Arnold b...,Überweisung an Regierungsrat,Gesundheitsdirektion,Motion,hängig,2019-09-10
7,3008,/gast/geschaefte/1996,Motion von Rainer Leemann und Michael Arnold b...,Überweisung an Regierungsrat,Gesundheitsdirektion,Motion,hängig,2019-09-10
8,3007,/gast/geschaefte/1995,Interpellation der SP-Fraktion betreffend Pest...,Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig,2019-09-08
9,3007,/gast/geschaefte/1995,Interpellation der SP-Fraktion betreffend Pest...,Überweisung an Regierungsrat,Baudirektion,Interpellation,hängig,2019-09-08


In [13]:
df_mit_dat.set_index("Einreichedatum", inplace=True)

NameError: name 'jt' is not defined